# Data Name <img align="right" src="../../resources/easi_logo.jpg">

_Work in progress_

#### Index
- [Overview](#Overview)
- [Setup (imports, dask, query)](#Setup)
- [Product definition (measurements, flags)](#Product-definition)
- [Create a mask (quality layer)](#Create-a-mask)
- [Define scaling (scale, offset)](#Define-scaling)
- [Select a data layer (process)](#Select-a-data-layer)
- [Visualise](#Visualise)
- [Appendix](#Appendix)

## Overview

Brief information

#### Data source and documentation

Context and links

#### Reference datasets

If applicable, e.g. google earth or aws earth

#### EASI pipeline

| Task | Summary |
|------|---------|
| Source | |
| Download | |
| Preprocess | |
| Format | |
| Prepare | |
| TODO | |

## Setup

#### Imports

In [ ]:
# Data tools
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime

# Datacube
import datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py
from datacube.utils.rio import configure_s3_access

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
hv.extension('bokeh', logo=False)

# Python
import sys, os, re

# Optional EASI tools
sys.path.append(os.path.expanduser('../../scripts'))
import notebook_utils

#### Dask

In [ ]:
cluster, client = notebook_utils.initialize_dask(workers=(1,2), use_gateway=True)
display(cluster)
display(client)

#### AWS configuration

In [ ]:
# Optional
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

#### ODC database

In [ ]:
# For development products:
#  - This is a development ODC database while we test and demo this product.
# CONF = """
# [datacube]
# db_hostname: v2-db-easihub-csiro-eks.cluster-ro-cvaedcg0qvwd.ap-southeast-2.rds.amazonaws.com
# db_database: 
# db_username: 
# db_password: 
# """
# from datacube.config import read_config, LocalConfig
# dc = datacube.Datacube(config=LocalConfig(read_config(CONF)), env='datacube')

dc = datacube.Datacube()

#### Example query

Change any of the parameters in the query object below to adjust the location, time, projection, or spatial resolution of the returned datasets.

Use the Explorer interface to check the temporal and spatial coverage for each product:
- https://explorer.csiro.easi-eo.solutions  + /product (when available)

In [ ]:
# Area name
min_longitude, max_longitude = (,)
min_latitude, max_latitude = (,)
min_date = ''
max_date = ''
product = ''

native_crs = ''

query = {
    'product': product,                     # Product name
    'x': (min_longitude, max_longitude),    # "x" axis bounds
    'y': (min_latitude, max_latitude),      # "y" axis bounds
    'time': (min_date, max_date),           # Any parsable date strings
    'output_crs': native_crs,               # EPSG code
    'resolution': (0.01, 0.01),             # Target resolution
    'group_by': 'solar_day',                # Scene ordering
    'dask_chunks': {'latitude': 2048, 'longitude': 2048},  # Dask chunks
}

In [ ]:
# Load data
data = dc.load(**query)

notebook_utils.heading(notebook_utils.xarray_object_size(data))
display(data)

# Calculate valid (not nodata) masks for each layer
valid_mask = masking.valid_data_mask(data)

## Product definition

Display the measurement definitions for the selected product.

Use `list_measurements` to show the details for a product, and `masking.describe_variable_flags` to show the flag definitions.

In [ ]:
# Measurements for the selected product
measurements = dc.list_measurements().loc[query['product']]

# Separate lists of measurement data names and flag names
data_names = measurements[ pd.isnull(measurements.flags_definition)].index
flag_names = measurements[pd.notnull(measurements.flags_definition)].index

# Select one for use below
flag_name = flag_names[0]

## Create a mask

In [ ]:
# Make flags image
flag_data = data[[flag_name]].where(valid_mask[flag_name]).persist()   # Dataset
display(flag_data)

In [ ]:
# These options manipulate the color map and colorbar to show the categories for this product
options = {
    'title': f'Flag data for: {query["product"]} ({flag_name})',
    'cmap': cc.rainbow,
    'colorbar': True,
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'tools': ['hover'],
}

# Set the Dataset CRS
plot_crs = native_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

quality_plot = flag_data.hvplot.image(
    x = 'longitude', y = 'latitude',         # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mode(),          # Datashader selects mode value, requires 'hv.Image'
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                          # Dataset CRS
    projection = ccrs.PlateCarree(),         # Output Projection (ccrs.PlateCarree() when coastline=True)
    coastline = '10m',                       # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist()

# display(quality_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(quality_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

In [ ]:
# Create mask layer

# If binary mask
good_pixel_flags = {}
mask = masking.make_mask(data[flag_name], **good_pixel_flags)

# If enumeration
# good_pixel_flags = ()
# mask = enum_to_bool(data[flag_name], good_pixel_flags)

## Define scaling

In [ ]:
# Define scaling

scale = 1.
offset = 0.

## Select a data layer

In [ ]:
# Select a layer and apply masking and scaling, then persist in dask

layer_name = ''

# Apply valid mask and good pixel mask
layer = data[[layer_name]].where(valid_mask[layer_name] & good_pixel_mask) * scale + offset
layer = layer.persist()

## Visualise

In [ ]:
# Generate a plot

options = {
    'title': f'{query["product"]}: {layer_name}',
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'cmap': cc.rainbow,
    'clim': (0, 1),                          # Limit the color range depending on the layer_name
    'colorbar': True,
    'tools': ['hover'],
}

# Set the Dataset CRS
plot_crs = native_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

layer_plot = layer.hvplot.image(
    x = 'x', y = 'y',                        # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mean(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                        # Dataset crs
    projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
    coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

## Appendix